In [2733]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [2734]:
# rfr file, rm file
file_name_rfr = r"..\..\Refinitiv\New rfr.xlsx"
file_name_rm = r"..\..\Refinitiv\New rm.xlsx"

#get the sorted fin file
# file_name_str = r"..\..\Refinitiv\Analysis\AU\AU_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Refinitiv\Analysis\CN\CN_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Refinitiv\Analysis\HK\HK_Refinitiv_fin_data_portfolio.xlsx"
# file_name_str = r"..\..\Refinitiv\Analysis\MY\MY_Refinitiv_fin_data_portfolio.xlsx"
file_name_str = r"..\..\Refinitiv\Analysis\SG\SG_Refinitiv_fin_data_portfolio.xlsx"



# indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'mv'
# indicator = 'roa'
indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm_no,,,,,,,,,,,,,,,,,,,,,
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.707107,11.568620,12.017363,0.634619,-0.707107,-7.071068e-01,10.801095,7.893590,-0.707107,NaN
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.707107,NaN,24.357731,NaN,NaN,0.000000e+00,6.932902,2.354634,-0.707107,NaN
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2019,18.956321,5.950305,18.583850,32.960784,110.579577,196.126724,...,-0.707107,3.855826,8.784869,6.970720,-0.707107,-2.357023e-01,4.419383,1.756878,0.707107,9.360309
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2020,20.163526,7.326079,21.149379,31.933677,154.942011,233.948625,...,-0.707107,NaN,8.362799,NaN,NaN,-1.110223e-16,2.870669,0.166325,0.707107,10.081763
14,China Aviation Oil (Singapore) Corporation Ltd,CNAO.SI,1,2021,25.214499,13.774284,34.274059,23.295721,472.100623,320.881872,...,-0.707107,NaN,1.798649,NaN,NaN,-7.071068e-01,9.659823,10.327521,-0.707107,12.253696


## Sorting the data by firm name and then date

In [2735]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'year'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm_no,,,,,,,,,,,,,,,,,,,,,
1,AEM Holdings Ltd,AEM.SI,8,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.459130,13.981730,16.324743,15.456103,-0.151591,-0.210635,27.389168,37.572848,0.308960,NaN
1,AEM Holdings Ltd,AEM.SI,8,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.528780,5.940570,18.705622,18.444906,-0.692064,0.064237,14.126625,8.400372,1.469059,NaN
1,AEM Holdings Ltd,AEM.SI,8,2019,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.086577,7.073184,13.736628,14.561529,-0.457606,-0.423594,17.446831,10.026366,1.148245,NaN
1,AEM Holdings Ltd,AEM.SI,8,2020,27.381327,23.196373,34.181488,21.183007,792.886556,491.368931,...,0.166205,2.581016,17.494781,18.753909,-0.795235,-0.006628,16.179751,12.640899,1.460142,13.687349
1,AEM Holdings Ltd,AEM.SI,8,2021,30.768458,24.611775,32.681267,35.000344,804.343977,861.420566,...,0.844262,12.238264,20.671748,18.021131,-0.467978,-0.242646,10.140624,5.701218,1.309775,15.262906
1,AEM Holdings Ltd,AEM.SI,8,2022,39.545171,29.218572,43.867555,44.805155,1281.747315,1309.142638,...,-0.652048,3.319671,20.190899,15.184893,-1.111053,-0.206321,13.202667,3.405021,1.054739,19.669425
4,Bumitama Agri Ltd,BUMI.SI,5,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005827,11.367127,13.154107,9.625199,-0.185656,-0.034290,10.367825,9.202317,-0.096828,NaN
4,Bumitama Agri Ltd,BUMI.SI,5,2018,NaN,NaN,NaN,NaN,NaN,NaN,...,0.539603,15.798456,13.464667,6.888307,0.338804,0.199227,5.898729,4.708877,0.475866,NaN
4,Bumitama Agri Ltd,BUMI.SI,5,2019,51.540230,50.917832,48.398559,57.652977,2464.349685,2935.564611,...,0.057587,10.355942,9.264575,6.499889,0.167906,0.121002,5.969361,3.727632,-0.274625,25.830616


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_top20. For example, at the end of August, after selecting the top 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the top 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_top 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [2736]:
"""
change data format
"""

# portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
#     'datetime64')
# portfolio_selection_sorted.dtypes


'\nchange data format\n'

In [2737]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\..\Refinitiv\Analysis_top\\" + file_name[:2] + "\\" + indicator
final_quantile_folder = "..\..\Refinitiv\Analysis\\" + file_name[:2] + "\\" + indicator


# quantile_path = final_quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
top_path = final_quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_top20_second_highest.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_second_highest.xlsx"

rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2]+"_rfr" )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2]+"_rm" )

try:
    os.makedirs(quantile_folder)
    os.makedirs(final_quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [2738]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.017620
1,2018,0.020130
2,2019,0.016570
3,2020,0.003050
4,2021,0.004420
5,mean_yr,0.012358


In [2739]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.116881
1,2018,-0.131644
2,2019,0.010238
3,2020,-0.098272
4,2021,0.108782
5,mean_yr,0.001197


In [2740]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.017620,0.116881
1,2018,0.020130,-0.131644
2,2019,0.016570,0.010238
3,2020,0.003050,-0.098272
4,2021,0.004420,0.108782
5,mean_yr,0.012358,0.001197


In [2741]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


52

In [2742]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [2743]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')
        

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [2744]:
all_sort_companies_data = pd.concat(all_quantile_rank)

# all_sort_companies_data.to_excel(
#     quantile_path, sheet_name="main")


In [2745]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,esg,e,s,g,es,eg,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm_no,,,,,,,,,,,,,,,,,,,,,
41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,1118.207325,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,2687.562948,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


In [2746]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret_ln')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


In [2747]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [2748]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [2749]:
# book = load_workbook(quantile_path)
# writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
# writer.book = book
# df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
# writer.close()

In [2750]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["year", "quantile_rank"])

all_sort_companies_data.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,41,Hotel Properties Ltd,HPPS.SI,4,2017,NaN,NaN,NaN,NaN,NaN,...,1.693572,6.853578,4.470617,-1.154204,-0.309949,4.979294,1.932443,0.329775,NaN,0
1,32,Great Eastern Holdings Ltd,GELA.SI,7,2017,32.771085,29.060755,34.766197,38.478261,1010.331945,...,0.093682,1.739115,1.428202,-1.152101,-0.966200,3.402568,5.755630,-0.353976,15.902443,0
2,38,Hong Leong Finance Ltd,HLSF.SI,7,2017,NaN,NaN,NaN,NaN,NaN,...,2.466058,1.739115,1.428202,0.508992,0.619949,3.402568,5.755630,-0.471385,NaN,0
3,71,Sheng Siong Group Ltd,SHEN.SI,5,2017,NaN,NaN,NaN,NaN,NaN,...,18.327547,13.154107,9.625199,0.537489,0.101688,10.367825,9.202317,0.782990,NaN,0
4,73,Singapore Airlines Ltd,SIAL.SI,3,2017,55.639475,59.621212,59.838752,45.077295,3567.658939,...,0.681292,9.300856,14.976229,-0.575550,-0.502838,5.883379,5.247748,-0.774353,27.568318,0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [2751]:

#set the query string
query_str = ''
query_str = 'quantile_rank == ' + str(quantile_rank-2)

sort_companies_data_top20per = all_sort_companies_data.query(query_str)

sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
26,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,32.129075,13.154107,9.625199,1.971384,0.427196,10.367825,9.202317,-0.511192,12.460052,2
27,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,21.733473,50.114864,52.656163,-0.538995,-0.595859,8.334040,6.106522,-0.185241,10.195363,2
28,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,NaN,36.448271,28.562728,NaN,-0.342376,2.841186,2.099187,-0.333152,21.158680,2
29,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,1.872156,9.300856,14.976229,-0.496033,-0.063566,5.883379,5.247748,-0.992250,31.623022,2
30,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,0.132058,9.300856,14.976229,-0.612223,-0.471549,5.883379,5.247748,0.419863,15.923960,2


In [2752]:
sort_companies_data_top20per.columns

Index(['firm_no', 'firm_name', 'tkr', 'sec_no', 'year', 'esg', 'e', 's', 'g',
       'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta_5y', 'beta_6m',
       'roa', 'vol_180d', 'mv', 'vol_90d', 'ret_ln', 'rfr', 'rm', 'ep',
       'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean', 'sp_sec_sd',
       'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z', 'value',
       'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value', 'quantile_rank'],
      dtype='object')

In [2753]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['AEM Holdings Ltd', 'Bumitama Agri Ltd', 'CSE Global Ltd', 'China Aviation Oil (Singapore) Corporation Ltd', 'China Everbright Water Ltd', 'Comfortdelgro Corporation Ltd', 'Cosco Shipping International (Singapore) Co Ltd', 'DBS Group Holdings Ltd', 'DFI Retail Group Holdings Ltd', 'Delfi Ltd', 'First Resources Ltd', 'Fraser and Neave Ltd', 'Genting Singapore Ltd', 'Golden Agri-Resources Ltd', 'Golden Energy and Resources Ltd', 'Great Eastern Holdings Ltd', 'Haw Par Corporation Ltd', 'Hong Leong Finance Ltd', 'Hotel Properties Ltd', 'Hutchison Port Holdings Trust', 'Japfa Ltd', 'Jardine Cycle & Carriage Ltd', 'Jardine Matheson Holdings Ltd', 'Keppel Corporation Ltd', 'Keppel Infrastructure Trust', 'Mandarin Oriental International Ltd', 'NetLink NBN Trust', 'Oversea-Chinese Banking Corporation Ltd', 'Q & M Dental Group (Singapore) Ltd', 'Raffles Medical Group Ltd', 'Riverstone Holdings Ltd', 'SATS Ltd', 'SBS Transit Ltd', 'SIA Engineering Company Ltd', 'Sembcorp Industries Ltd', 'Sembco

In [2754]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret_ln'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


6
sum_esg_sray_dic: {2017: 349.0021976043762, 2018: 349.0021976043762, 2019: 349.0021976043762, 2020: 349.0021976043762, 2021: 349.0021976043762, 2022: 158.7109425242106, 2023: 0.0}
sum_ret_dict: {2017: -0.5754632413870076, 2018: -2.4479767313217637, 2019: 1.8191991993589227, 2020: -0.9729495500717927, 2021: 11.409780958765118, 2022: 8.30913560116624, 2023: 0.0}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [2755]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2017: 0.07692307692307693, 2018: 0.07692307692307693, 2019: 0.07692307692307693, 2020: 0.07692307692307693, 2021: 0.07692307692307693, 2022: 0.125}
no_of_companies: {2017: 13.0, 2018: 13.0, 2019: 13.0, 2020: 13.0, 2021: 13.0, 2022: 8.0}


### total companies from all quantile

In [2756]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 52.0, 2018: 52.0, 2019: 52.0, 2020: 52.0, 2021: 52.0, 2022: 34.0}


## Calculate the number of companies (top 20%) based on the date

In [2757]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(top 20)']
no_of_companies_df.head()

,year,no of firms(top 20)
0,2017,13.0
1,2018,13.0
2,2019,13.0
3,2020,13.0
4,2021,13.0


## Calculate the number of companies based on the date

In [2758]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,52.0
1,2018,52.0
2,2019,52.0
3,2020,52.0
4,2021,52.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [2759]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2017,0.076923
1,2018,0.076923
2,2019,0.076923
3,2020,0.076923
4,2021,0.076923


In [2760]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_top20.xlsx")

In [2761]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,-0.575463
1,2018,-2.447977
2,2019,1.819199
3,2020,-0.972950
4,2021,11.409781


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [2762]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_vol_180d
0,2017,349.002198
1,2018,349.002198
2,2019,349.002198
3,2020,349.002198
4,2021,349.002198


### Merging with the data of top 20% **only**

In [2763]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    equal_weight_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_ret_df, on='year')
sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,1.971384,0.427196,10.367825,9.202317,-0.511192,12.460052,2,0.076923,349.002198,-0.575463
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-0.538995,-0.595859,8.334040,6.106522,-0.185241,10.195363,2,0.076923,349.002198,-0.575463
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,NaN,-0.342376,2.841186,2.099187,-0.333152,21.158680,2,0.076923,349.002198,-0.575463
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-0.496033,-0.063566,5.883379,5.247748,-0.992250,31.623022,2,0.076923,349.002198,-0.575463
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-0.612223,-0.471549,5.883379,5.247748,0.419863,15.923960,2,0.076923,349.002198,-0.575463


### Calculate the factor weight

In [2764]:
if sort_companies_data_top20per[indicator].empty and sort_companies_data_top20per['sum_'+indicator].empty:
    sort_companies_data_top20per['fc_wg'] = None
else:
    sort_companies_data_top20per['fc_wg'] = sort_companies_data_top20per[indicator] / \
        sort_companies_data_top20per['sum_'+indicator]

sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,0.427196,10.367825,9.202317,-0.511192,12.460052,2,0.076923,349.002198,-0.575463,0.060849
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-0.595859,8.334040,6.106522,-0.185241,10.195363,2,0.076923,349.002198,-0.575463,0.065196
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,-0.342376,2.841186,2.099187,-0.333152,21.158680,2,0.076923,349.002198,-0.575463,0.065414
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-0.063566,5.883379,5.247748,-0.992250,31.623022,2,0.076923,349.002198,-0.575463,0.065935
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-0.471549,5.883379,5.247748,0.419863,15.923960,2,0.076923,349.002198,-0.575463,0.070236


In [2765]:
sort_companies_data_top20per['r'] = (
    sort_companies_data_top20per['ret_ln'] - sort_companies_data_top20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_top20per['eq_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['fc_wg'])

"""
ret
"""
sort_companies_data_top20per['eq_wg_ret'] = (
    sort_companies_data_top20per['ret_ln'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_ret'] = (
    sort_companies_data_top20per['ret_ln'] * sort_companies_data_top20per['fc_wg'])

sort_companies_data_top20per.head(15)


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,quantile_rank,eq_wg,sum_vol_180d,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,2,0.076923,349.002198,-0.575463,0.060849,-0.295252,-0.022712,-0.017966,-0.021356,-0.016894
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,2,0.076923,349.002198,-0.575463,0.065196,-0.035639,-0.002741,-0.002323,-0.001386,-0.001175
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,2,0.076923,349.002198,-0.575463,0.065414,-0.194019,-0.014925,-0.012692,-0.013569,-0.011539
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,2,0.076923,349.002198,-0.575463,0.065935,-0.228537,-0.017580,-0.015069,-0.016224,-0.013907
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,2,0.076923,349.002198,-0.575463,0.070236,-0.178393,-0.013723,-0.012530,-0.012367,-0.011292
5,15,Cosco Shipping International (Singapore) Co Ltd,COSC.SI,3,2017,13.735068,4.689992,25.952042,8.707094,121.714869,...,2,0.076923,349.002198,-0.575463,0.077607,-0.017620,-0.001355,-0.001367,0.000000,0.000000
6,43,Jardine Matheson Holdings Ltd,JARD.SI,3,2017,10.658215,16.548295,8.900869,4.880000,147.294206,...,2,0.076923,349.002198,-0.575463,0.078623,-0.241914,-0.018609,-0.019020,-0.017253,-0.017635
7,17,CSE Global Ltd,CSES.SI,8,2017,37.532517,34.485651,49.498380,28.142639,1706.983861,...,2,0.076923,349.002198,-0.575463,0.082522,0.352127,0.027087,0.029058,0.028442,0.030512
8,31,Golden Agri-Resources Ltd,GAGR.SI,5,2017,50.640579,59.441572,62.735123,20.062802,3729.074312,...,2,0.076923,349.002198,-0.575463,0.083555,-0.059293,-0.004561,-0.004954,-0.003206,-0.003482
9,62,Q & M Dental Group (Singapore) Ltd,QMDT.SI,6,2017,NaN,NaN,NaN,NaN,NaN,...,2,0.076923,349.002198,-0.575463,0.085500,-0.058442,-0.004496,-0.004997,-0.003140,-0.003490


## Excess return portfolio
### Equal Weight and Factor Weight

In [2766]:
initial_year = 2017
stop_year = 2023

eq_wg_r_p_dict = {}
fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        r
        """
        eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        eq_wg_r_p_dict.setdefault(year, eq_wg_r_p)
        fc_wg_r_p_dict.setdefault(year, fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"eq_wg_r_p_dict: {eq_wg_r_p_dict}")
print(f"fc_wg_r_p_dict: {fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


eq_wg_r_p_dict: {2017: -0.06188640318361599, 2018: -0.20843590240936644, 2019: 0.12336839995068637, 2020: -0.0778922730824456, 2021: 0.8739354583665475, 2022: 0.0, 2023: 0.0}
fc_wg_r_p_dict: {2017: -0.04813286267780943, 2018: -0.21260539397269632, 2019: 0.1337630580560416, 2020: -0.09619151972997557, 2021: 0.9294208067349728, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: -0.044266403183615965, 2018: -0.18830590240936643, 2019: 0.13993839995068638, 2020: -0.07484227308244559, 2021: 0.8776754583665476, 2022: 1.03864195014578, 2023: 0.0}
fc_wg_ret_p_dict: {2017: -0.030512862677809432, 2018: -0.1924753939726963, 2019: 0.1503330580560416, 2020: -0.09314151972997559, 2021: 0.9331057407748686, 2022: 1.052026717255284, 2023: 0.0}


In [2767]:
eq_wg_r_p_dict_items = eq_wg_r_p_dict.items()
eq_wg_r_p_dict_list = list(eq_wg_r_p_dict_items)

eq_wg_r_p_dict_df = pd.DataFrame(eq_wg_r_p_dict_list)
eq_wg_r_p_dict_df.columns = ['year', 'eq_wg_r_p']
eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
eq_wg_r_p_dict_df.head()

,year,eq_wg_r_p
0,2017,-0.061886
1,2018,-0.208436
2,2019,0.123368
3,2020,-0.077892
4,2021,0.873935


In [2768]:
fc_wg_r_p_dict_items = fc_wg_r_p_dict.items()
fc_wg_r_p_dict_list = list(fc_wg_r_p_dict_items)

fc_wg_r_p_dict_df = pd.DataFrame(fc_wg_r_p_dict_list)
fc_wg_r_p_dict_df.columns = ['year', 'fc_wg_r_p']
fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_r_p_dict_df.head()


,year,fc_wg_r_p
0,2017,-0.048133
1,2018,-0.212605
2,2019,0.133763
3,2020,-0.096192
4,2021,0.929421


In [2769]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,-0.044266
1,2018,-0.188306
2,2019,0.139938
3,2020,-0.074842
4,2021,0.877675


In [2770]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,-0.030513
1,2018,-0.192475
2,2019,0.150333
3,2020,-0.093142
4,2021,0.933106


In [2771]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,52.0,13.0,-0.061886,-0.048133,-0.044266,-0.030513
1,2018,52.0,13.0,-0.208436,-0.212605,-0.188306,-0.192475
2,2019,52.0,13.0,0.123368,0.133763,0.139938,0.150333
3,2020,52.0,13.0,-0.077892,-0.096192,-0.074842,-0.093142
4,2021,52.0,13.0,0.873935,0.929421,0.877675,0.933106


### Merging the eq and fc wg of the r_p into the big dataframe

In [2772]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_r_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_r_p_dict_df, on='year')

sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_top20per.head(15)


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,0.060849,-0.295252,-0.022712,-0.017966,-0.021356,-0.016894,-0.061886,-0.048133,-0.044266,-0.030513
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,0.065196,-0.035639,-0.002741,-0.002323,-0.001386,-0.001175,-0.061886,-0.048133,-0.044266,-0.030513
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,0.065414,-0.194019,-0.014925,-0.012692,-0.013569,-0.011539,-0.061886,-0.048133,-0.044266,-0.030513
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,0.065935,-0.228537,-0.017580,-0.015069,-0.016224,-0.013907,-0.061886,-0.048133,-0.044266,-0.030513
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,0.070236,-0.178393,-0.013723,-0.012530,-0.012367,-0.011292,-0.061886,-0.048133,-0.044266,-0.030513
5,15,Cosco Shipping International (Singapore) Co Ltd,COSC.SI,3,2017,13.735068,4.689992,25.952042,8.707094,121.714869,...,0.077607,-0.017620,-0.001355,-0.001367,0.000000,0.000000,-0.061886,-0.048133,-0.044266,-0.030513
6,43,Jardine Matheson Holdings Ltd,JARD.SI,3,2017,10.658215,16.548295,8.900869,4.880000,147.294206,...,0.078623,-0.241914,-0.018609,-0.019020,-0.017253,-0.017635,-0.061886,-0.048133,-0.044266,-0.030513
7,17,CSE Global Ltd,CSES.SI,8,2017,37.532517,34.485651,49.498380,28.142639,1706.983861,...,0.082522,0.352127,0.027087,0.029058,0.028442,0.030512,-0.061886,-0.048133,-0.044266,-0.030513
8,31,Golden Agri-Resources Ltd,GAGR.SI,5,2017,50.640579,59.441572,62.735123,20.062802,3729.074312,...,0.083555,-0.059293,-0.004561,-0.004954,-0.003206,-0.003482,-0.061886,-0.048133,-0.044266,-0.030513
9,62,Q & M Dental Group (Singapore) Ltd,QMDT.SI,6,2017,NaN,NaN,NaN,NaN,NaN,...,0.085500,-0.058442,-0.004496,-0.004997,-0.003140,-0.003490,-0.061886,-0.048133,-0.044266,-0.030513


In [2773]:
sort_companies_data_top20per['rm_r_y1'] = (
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']) * 12
sort_companies_data_top20per['rm_r_y2'] = (((1+(
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr'])) ** 12) - 1)

sort_companies_data_top20per['eq_wg_r_p_y1'] = sort_companies_data_top20per['eq_wg_r_p'] *12
sort_companies_data_top20per['eq_wg_r_p_y2'] = (((1 + sort_companies_data_top20per['eq_wg_r_p'])**12) - 1)
sort_companies_data_top20per['eq_r_alpha_y1'] = sort_companies_data_top20per['eq_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_r_alpha_y2'] = sort_companies_data_top20per['eq_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,-0.061886,-0.048133,-0.044266,-0.030513,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-0.061886,-0.048133,-0.044266,-0.030513,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,-0.061886,-0.048133,-0.044266,-0.030513,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-0.061886,-0.048133,-0.044266,-0.030513,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-0.061886,-0.048133,-0.044266,-0.030513,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623


In [2774]:
sort_companies_data_top20per['fc_wg_r_p_y1'] = sort_companies_data_top20per['fc_wg_r_p'] * 12
sort_companies_data_top20per['fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_r_p'])**12) - 1)
sort_companies_data_top20per['fc_r_alpha_y1'] = sort_companies_data_top20per['fc_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_r_alpha_y2'] = sort_companies_data_top20per['fc_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,1.191128,2.113208,-0.742637,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965


In [2775]:

# sort_companies_data_top20per.to_excel(
#     "../3.04/SG_esg_top20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [2776]:
sort_companies_data_top20per['beta_5y'] = sort_companies_data_top20per['beta_5y'].replace("-", np.nan)
sort_companies_data_top20per['beta_5y'] = sort_companies_data_top20per['beta_5y'].astype(float)
sort_companies_data_top20per['rc'] = sort_companies_data_top20per['rfr'] + \
    (sort_companies_data_top20per['beta_5y'] *
    (sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_top20per['eq_wg_rc_int'] = sort_companies_data_top20per ['eq_wg'] * sort_companies_data_top20per['rc']

"""
factor weight rc
"""
sort_companies_data_top20per['fc_wg_rc_int'] = sort_companies_data_top20per['fc_wg'] * \
    sort_companies_data_top20per['rc']

sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.059776,0.004598,0.003637
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.092381,0.007106,0.006023
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.118381,0.009106,0.007744
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.127013,0.009770,0.008375
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-0.535415,-1.933765,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.114955,0.008843,0.008074


In [2777]:
initial_year = 2017
stop_year = 2023

eq_wg_rc_p_dict = {}
fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        rc
        """
        eq_wg_rc_p = each_year_companies['eq_wg_rc_int'].sum()
        fc_wg_rc_p = each_year_companies['fc_wg_rc_int'].sum()

        eq_wg_rc_p_dict.setdefault(year, eq_wg_rc_p)
        fc_wg_rc_p_dict.setdefault(year, fc_wg_rc_p)


print(f"eq_wg_rc_p_dict: {eq_wg_rc_p_dict}")
print(f"fc_wg_rc_p_dict: {fc_wg_rc_p_dict}")


eq_wg_rc_p_dict: {2017: 0.10934962360442267, 2018: -0.12012883525591173, 2019: 0.010718383553347775, 2020: -0.09058461375617936, 2021: 0.10086386843831825, 2022: 0.0, 2023: 0.0}
fc_wg_rc_p_dict: {2017: 0.1094263969404786, 2018: -0.12024622523605981, 2019: 0.010713486028444952, 2020: -0.09066298147846906, 2021: 0.10094458737328389, 2022: 0.0, 2023: 0.0}


In [2778]:
eq_wg_rc_p_dict_items = eq_wg_rc_p_dict.items()
eq_wg_rc_p_dict_list = list(eq_wg_rc_p_dict_items)

eq_wg_rc_p_dict_df = pd.DataFrame(eq_wg_rc_p_dict_list)
eq_wg_rc_p_dict_df.columns = ['year', 'eq_wg_rc_p']
eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_rc_p_dict_df.head()


,year,eq_wg_rc_p
0,2017,0.109350
1,2018,-0.120129
2,2019,0.010718
3,2020,-0.090585
4,2021,0.100864


In [2779]:
fc_wg_rc_p_dict_items = fc_wg_rc_p_dict.items()
fc_wg_rc_p_dict_list = list(fc_wg_rc_p_dict_items)

fc_wg_rc_p_dict_df = pd.DataFrame(fc_wg_rc_p_dict_list).dropna()
fc_wg_rc_p_dict_df.columns = ['year', 'fc_wg_rc_p']
fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_rc_p_dict_df.head()


,year,fc_wg_rc_p
0,2017,0.109426
1,2018,-0.120246
2,2019,0.010713
3,2020,-0.090663
4,2021,0.100945


In [2780]:
SG_esg_int = pd.merge(SG_esg_int, eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p
0,2017,52.0,13.0,-0.061886,-0.048133,-0.044266,-0.030513,0.109350,0.109426
1,2018,52.0,13.0,-0.208436,-0.212605,-0.188306,-0.192475,-0.120129,-0.120246
2,2019,52.0,13.0,0.123368,0.133763,0.139938,0.150333,0.010718,0.010713
3,2020,52.0,13.0,-0.077892,-0.096192,-0.074842,-0.093142,-0.090585,-0.090663
4,2021,52.0,13.0,0.873935,0.929421,0.877675,0.933106,0.100864,0.100945


In [2781]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_rc_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_rc_p_dict_df, on='year')

sort_companies_data_top20per.head(15)

,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.059776,0.004598,0.003637,0.109350,0.109426
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.092381,0.007106,0.006023,0.109350,0.109426
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.118381,0.009106,0.007744,0.109350,0.109426
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.127013,0.009770,0.008375,0.109350,0.109426
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.114955,0.008843,0.008074,0.109350,0.109426
5,15,Cosco Shipping International (Singapore) Co Ltd,COSC.SI,3,2017,13.735068,4.689992,25.952042,8.707094,121.714869,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.220727,0.016979,0.017130,0.109350,0.109426
6,43,Jardine Matheson Holdings Ltd,JARD.SI,3,2017,10.658215,16.548295,8.900869,4.880000,147.294206,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.092847,0.007142,0.007300,0.109350,0.109426
7,17,CSE Global Ltd,CSES.SI,8,2017,37.532517,34.485651,49.498380,28.142639,1706.983861,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.109294,0.008407,0.009019,0.109350,0.109426
8,31,Golden Agri-Resources Ltd,GAGR.SI,5,2017,50.640579,59.441572,62.735123,20.062802,3729.074312,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.116647,0.008973,0.009746,0.109350,0.109426
9,62,Q & M Dental Group (Singapore) Ltd,QMDT.SI,6,2017,NaN,NaN,NaN,NaN,NaN,...,-2.648623,-0.577594,-0.446757,-1.768722,-2.559965,0.077800,0.005985,0.006652,0.109350,0.109426


In [2782]:
sort_companies_data_top20per['eq_wg_rc_p_y1'] = sort_companies_data_top20per['eq_wg_rc_p'] * 12
sort_companies_data_top20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['eq_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['eq_rc_alpha_y1'] = sort_companies_data_top20per['eq_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_rc_alpha_y2'] = sort_companies_data_top20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,-2.559965,0.059776,0.004598,0.003637,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,-2.559965,0.092381,0.007106,0.006023,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,-2.559965,0.118381,0.009106,0.007744,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,-2.559965,0.127013,0.009770,0.008375,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,-2.559965,0.114955,0.008843,0.008074,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724


In [2783]:
sort_companies_data_top20per['fc_wg_rc_p_y1'] = sort_companies_data_top20per['fc_wg_rc_p'] * 12
sort_companies_data_top20per['fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['fc_rc_alpha_y1'] = sort_companies_data_top20per['fc_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_rc_alpha_y2'] = sort_companies_data_top20per['fc_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm_no,firm_name,tkr,sec_no,year,esg,e,s,g,es,...,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,fc_wg_rc_p_y1,fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,20,Delfi Ltd,DELF.SI,5,2017,24.492909,14.362031,24.895070,34.942029,357.543772,...,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724,1.313117,2.476818,0.121989,0.36361
1,63,Raffles Medical Group Ltd,RAFG.SI,6,2017,20.986586,19.743590,20.816993,21.811594,411.002179,...,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724,1.313117,2.476818,0.121989,0.36361
2,68,Sembcorp Industries Ltd,SCIL.SI,10,2017,42.659736,47.837986,37.348478,40.951691,1786.675961,...,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724,1.313117,2.476818,0.121989,0.36361
3,48,Keppel Corporation Ltd,KPLM.SI,3,2017,63.309610,51.414125,62.453573,82.149758,3210.995794,...,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724,1.313117,2.476818,0.121989,0.36361
4,44,Jardine Cycle & Carriage Ltd,JCYC.SI,3,2017,32.319470,17.617549,48.321257,25.610221,851.302103,...,0.10935,0.109426,1.312195,2.473932,0.121068,0.360724,1.313117,2.476818,0.121989,0.36361


In [2784]:
SG_int_wanted = sort_companies_data_top20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'eq_wg_r_p_y2', 'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'fc_wg_r_p_y2', 'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,eq_wg_r_p_y2,fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2017,1.191128,2.113208,-0.742637,-1.933765,-2.648623,-0.535415,-0.577594,-1.768722,-2.559965,-0.446757,1.312195,0.121068,0.360724,2.473932,1.313117,0.121989,0.363610,2.476818
13,2018,-1.821289,-0.861280,-2.501231,-0.679941,-0.078209,-0.939489,-2.551265,-0.729975,-0.081925,-0.943205,-1.441546,0.379743,0.076573,-0.784707,-1.442955,0.378335,0.076228,-0.785052
26,2019,-0.075984,-0.073393,1.480421,1.556405,3.112324,3.038931,1.605157,1.681141,3.584339,3.510946,0.128621,0.204605,0.209874,0.136480,0.128562,0.204546,0.209808,0.136414
39,2020,-1.215864,-0.722509,-0.934707,0.281157,0.100412,-0.622097,-1.154298,0.061566,0.019619,-0.702890,-1.087015,0.128849,0.042507,-0.680002,-1.087956,0.127909,0.042176,-0.680333
52,2021,1.252343,2.291070,10.487226,9.234882,1871.943270,1874.234340,11.153050,9.900706,2658.190673,2660.481743,1.210366,-0.041977,-0.122937,2.168133,1.211335,-0.041008,-0.120148,2.170922


In [2785]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2017,52.0,13.0,-0.061886,-0.048133,-0.044266,-0.030513,0.109350,0.109426,1.191128,...,-2.559965,-0.446757,1.312195,0.121068,0.360724,2.473932,1.313117,0.121989,0.363610,2.476818
1,2018,52.0,13.0,-0.208436,-0.212605,-0.188306,-0.192475,-0.120129,-0.120246,-1.821289,...,-0.081925,-0.943205,-1.441546,0.379743,0.076573,-0.784707,-1.442955,0.378335,0.076228,-0.785052
2,2019,52.0,13.0,0.123368,0.133763,0.139938,0.150333,0.010718,0.010713,-0.075984,...,3.584339,3.510946,0.128621,0.204605,0.209874,0.136480,0.128562,0.204546,0.209808,0.136414
3,2020,52.0,13.0,-0.077892,-0.096192,-0.074842,-0.093142,-0.090585,-0.090663,-1.215864,...,0.019619,-0.702890,-1.087015,0.128849,0.042507,-0.680002,-1.087956,0.127909,0.042176,-0.680333
4,2021,52.0,13.0,0.873935,0.929421,0.877675,0.933106,0.100864,0.100945,1.252343,...,2658.190673,2660.481743,1.210366,-0.041977,-0.122937,2.168133,1.211335,-0.041008,-0.120148,2.170922


In [2786]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(top 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'eq_wg_r_p',
                            'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_wg_r_p_y2', 'eq_r_alpha_y2', 'fc_wg_r_p',
                            'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_wg_r_p_y2', 'fc_r_alpha_y2', 'eq_wg_rc_p',
                            'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'fc_wg_rc_p',
                            'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,52.0,13.0,1.191128,2.113208,-0.044266,-0.030513,-0.061886,-0.742637,-1.933765,...,0.109350,1.312195,0.121068,2.473932,0.360724,0.109426,1.313117,0.121989,2.476818,0.363610
1,2018,52.0,13.0,-1.821289,-0.861280,-0.188306,-0.192475,-0.208436,-2.501231,-0.679941,...,-0.120129,-1.441546,0.379743,-0.784707,0.076573,-0.120246,-1.442955,0.378335,-0.785052,0.076228
2,2019,52.0,13.0,-0.075984,-0.073393,0.139938,0.150333,0.123368,1.480421,1.556405,...,0.010718,0.128621,0.204605,0.136480,0.209874,0.010713,0.128562,0.204546,0.136414,0.209808
3,2020,52.0,13.0,-1.215864,-0.722509,-0.074842,-0.093142,-0.077892,-0.934707,0.281157,...,-0.090585,-1.087015,0.128849,-0.680002,0.042507,-0.090663,-1.087956,0.127909,-0.680333,0.042176
4,2021,52.0,13.0,1.252343,2.291070,0.877675,0.933106,0.873935,10.487226,9.234882,...,0.100864,1.210366,-0.041977,2.168133,-0.122937,0.100945,1.211335,-0.041008,2.170922,-0.120148


## Average of the int_reporting

In [2787]:
SG_esg_int['year'] = SG_esg_int['year'].astype(float)
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
2,2019.0,52.0,13.000000,-0.075984,-0.073393,0.139938,0.150333,0.123368,1.480421,1.556405,...,0.010718,0.128621,0.204605,0.136480,0.209874,0.010713,0.128562,0.204546,0.136414,0.209808
3,2020.0,52.0,13.000000,-1.215864,-0.722509,-0.074842,-0.093142,-0.077892,-0.934707,0.281157,...,-0.090585,-1.087015,0.128849,-0.680002,0.042507,-0.090663,-1.087956,0.127909,-0.680333,0.042176
4,2021.0,52.0,13.000000,1.252343,2.291070,0.877675,0.933106,0.873935,10.487226,9.234882,...,0.100864,1.210366,-0.041977,2.168133,-0.122937,0.100945,1.211335,-0.041008,2.170922,-0.120148
5,2022.0,34.0,8.000000,NaN,NaN,1.038642,1.052027,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,49.0,12.166667,-0.133933,0.549419,0.291474,0.303223,0.129818,1.557814,1.691748,...,0.002044,0.024524,0.158458,0.662767,0.113348,0.002035,0.024421,0.158354,0.663754,0.114335


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [2788]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,eq_wg_r_p_y2,eq_r_alpha_y2,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,-0.215872,0.798251,0.742434,0.756885,0.671244,0.671244,0.858433,1.000626,1.000322,...,0.043151,0.043151,2.316950,0.950619,1.391463,0.042933,0.042933,2.328235,0.951016,1.402620
1,p-value,0.839649,0.469433,0.499056,0.491254,0.538836,0.538836,0.439049,0.373632,0.373763,...,0.967649,0.967649,0.081409,0.395628,0.236475,0.967813,0.967813,0.080407,0.395449,0.233377


In [2789]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
4,2021.0,52.0,13.000000,1.252343,2.291070,0.877675,0.933106,0.873935,10.487226,9.234882,...,0.100864,1.210366,-0.041977,2.168133,-0.122937,0.100945,1.211335,-0.041008,2.170922,-0.120148
5,2022.0,34.0,8.000000,NaN,NaN,1.038642,1.052027,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,49.0,12.166667,-0.133933,0.549419,0.291474,0.303223,0.129818,1.557814,1.691748,...,0.002044,0.024524,0.158458,0.662767,0.113348,0.002035,0.024421,0.158354,0.663754,0.114335
0,t-score,NaN,NaN,-0.215872,0.798251,0.742434,0.756885,0.671244,0.671244,0.858433,...,0.043151,0.043151,2.316950,0.950619,1.391463,0.042933,0.042933,2.328235,0.951016,1.402620
1,p-value,NaN,NaN,0.839649,0.469433,0.499056,0.491254,0.538836,0.538836,0.439049,...,0.967649,0.967649,0.081409,0.395628,0.236475,0.967813,0.967813,0.080407,0.395449,0.233377


### Export the result into excel file

In [2790]:
sort_companies_data_top20per.to_excel(top_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
